In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from scipy.fft import fft2
from scipy.ndimage import gaussian_filter
from skimage.filters import unsharp_mask
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
import cv2
import numpy as np


In [41]:
color_image = cv2.imread('imgNocturna.png').astype(np.float64)

# Parameters
cutoff = 10 #frecuencia a la cual se produce la atenuación
order = 2 #atenuación sea más o menos pronunciada
Yh = 0.0999 
Yl = 1.01

homomorphic_image = np.zeros_like(color_image)

for i in range(color_image.shape[2]):
    color_channel = color_image[:, :, i]

    # Add a small offset to avoid division by zero
    epsilon = 1e-8
    color_channel_offset = color_channel + epsilon

    # Calculate the logarithm
    color_channel_logued = np.log(1 + color_channel_offset)

    # DFT
    color_channel_dft = np.fft.fft2(color_channel_logued)

    # Filter
    x, y = color_channel.shape
    A = np.zeros((x, y))
    H = np.zeros((x, y))
    d = cutoff
    n = order

    for j in range(x):
        for k in range(y):
            A[j, k] = np.sqrt((j - x / 2) ** 2 + (k - y / 2) ** 2)
            if A[j, k] == 0:
                H[j, k] = 0
            else:
                H[j, k] = 1 / (1 + ((d / A[j, k]) ** (2 * n)))

    H = ((Yl - Yh) * H) + Yh
    H = 1 - H

    color_channel_f = H * color_channel_dft

    # Inverse DFT
    color_channel_n = np.abs(np.fft.ifft2(color_channel_f))

    # Inverse log
    color_channel_final = np.exp(color_channel_n)

    # Update the original color channel with the processed values
    homomorphic_image[:, :, i] = color_channel_final

# Normalize and convert to uint8
homomorphic_image = ((homomorphic_image - np.min(homomorphic_image)) / (np.max(homomorphic_image) - np.min(homomorphic_image)) * 255).astype(np.uint8)

#Normalizar y suavizar:

# Save the result
cv2.imwrite('homomorphicColor1.png', homomorphic_image) #Homographic es del TP3

True

In [38]:
# Cargar la imagen en color
image = cv2.imread('imgNocturna.png')

# Convertir la imagen a escala de grises
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Oscurecer la imagen en escala de grises
factor_oscurecimiento = 0.4
darkened_gray_image = (gray_image * factor_oscurecimiento).astype(np.uint8)

# Valor a usar para el high-boost
A = 1.5

mask = np.array([[-1, -1, -1],
                [-1, A + 8, -1],
                [-1, -1, -1]], dtype=np.float32)

# Aplicar el filtro Laplaciano
hb_laplacian = cv2.filter2D(darkened_gray_image, -1, mask)

# Sumar el resultado al canal original
hb_image = cv2.addWeighted(image, 1.0, cv2.cvtColor(hb_laplacian, cv2.COLOR_GRAY2BGR), 1.0, 0)

# Para limitar los valores máximos de brillo (>255 ya satura)
hb_image = np.clip(hb_image, 0, 255).astype(np.uint8)

# Guardar las imágenes
cv2.imwrite('results/original_image.png', image)
cv2.imwrite('results/high_boosted_image_nocturna.png', hb_image) #HIGH BOOST TP3

True

In [46]:
# Aplicar suavizado 

# Cargar la imagen en color
image = cv2.imread('homomorphicColor1.png')
homomorphic_image_smoothed =  cv2.blur(image, (3, 3))

# Guardar el resultado
cv2.imwrite('homomorphicColor1_smoothed.png', homomorphic_image_smoothed)

True

In [6]:
#Filtro pasa alto tp2--> para arbol navidad
from skimage import io, color, img_as_ubyte
from skimage.filters import unsharp_mask

# Cargar la imagen a color
image = io.imread('homomorphicColor1.png')

# Aplicar unsharp mask a cada canal de color
sharpened_channels = [unsharp_mask(channel, radius=1, amount=1.5) for channel in image.transpose(2, 0, 1)]

# Combinar los canales
sharpened_image = np.array(sharpened_channels).transpose(1, 2, 0)

# Convertir la imagen de vuelta a formato de 8 bits
sharpened_image = img_as_ubyte(sharpened_image)

# Guardar la imagen mejorada
io.imsave('sharpened_homotphic1.png', sharpened_image)


In [8]:
from PIL import Image, ImageEnhance

# Open the image
image = Image.open('high_boosted_image_nocturna.png')

# Create an ImageEnhance object and adjust the contrast
# (factor > 1 increases contrast, factor < 1 decreases contrast)
factor = 1.5  # You can adjust this value
enhancer = ImageEnhance.Contrast(image)
adjusted_image = enhancer.enhance(factor)


# Save the adjusted image
adjusted_image.save(f'results/high_boosted_image_nocturna_contrast{factor}.jpg')


In [9]:
#corrección gamma tp color
def apply_gamma_correction(image, gamma=1.0):
    # Normalizar los valores de píxeles al rango [0, 1]
    normalized_image = image / 255.0
    
    # Aplicar la corrección gamma
    gamma_corrected_image = np.power(normalized_image, 1.0 / gamma)
    
    # Escalar nuevamente al rango [0, 255]
    gamma_corrected_image = (gamma_corrected_image * 255).astype(np.uint8)

    return gamma_corrected_image

# Cargar la imagen
image = cv2.imread('high_boosted_image_2.png')

# Aplicar corrección gamma con un valor específico (puedes ajustar este valor)
gamma_value = 1.5
gamma_corrected_image = apply_gamma_correction(image, gamma=gamma_value)

cv2.imwrite('high_boosted_image_2_correccion_gamma.png', gamma_corrected_image)

True

In [ ]:
#interpolacion? 

In [ ]:
#HDR?	